# 概要
- 下記の書籍の再現
  - https://book.mynavi.jp/ec/products/detail/id=115805
  - 4章 P74-80
  - 上記に加えて、回帰分析ができるパッケージはsklearn・statsmodels・pingouinの3つがあるので、それぞれで実行して結果に差異がないか確認する
- 分析の目的はテレビCMの視聴有無が購入量に与える影響を明らかにする
- 下記のサンプルデータを作成する
  - 年齢が高く女性のほうがテレビをよく見る
  - 年齢が低く男性のほうが購入量が多い
  - 真の平均介入効果を10（テレビCMを見たほうが購入量が10高い）
- データ概要
  - 結果変数：購入量
  - 原因変数：テレビCM視聴
  - 共変量:年齢、性別

# 使用するパッケージ（ライブラリと関数）を定義

In [1]:
import random
import numpy as np

# 乱数のシート値を固定
np.random.seed(1234)
random.seed(1234)

In [2]:
# 標準正規分布の生成用
from numpy.random import *

# グラフの描画用
import matplotlib.pyplot as plt

# SciPy 平均0、分散1に正規化（標準化）関数
import scipy.stats

# シグモイド関数
from scipy.special import expit

# その他
import pandas as pd

# サンプルデータ作成

### 原因変数と共変量作成

In [3]:
# サンプルサイズ
num_data = 200

# 年齢
x_1=randint(15, 76, num_data) # 15歳から75際の一様乱数

# 性別（0を女性、1を男性とする）
x_2=randint(0, 2, num_data) # 0 or 1の乱数

# ノイズの生成（平均0、標準偏差1の正規分布[ガウス分布]）
e_z=randn(num_data)

### 結果変数の作成

In [4]:
# シグモイド関数に入れる部分
z_base=x_1 + (1-x_2)*10 - 40 + 5*e_z
## 年齢が高くなると1の確率が高くなる
## 女性のほうが1の確率が高くなる[(1-x_2)*10の部分]
## z_baseはシグモイド関数のx（横軸）の値

# シグモイド関数を計算
z_prob=expit(0.1*z_base)
## z_probには0から1の間の数が入る
## 係数aは0.1

# テレビCMを見たかどうかの変数(0は見ていない、1は見た)
Z=np.array([])

for i in range(num_data):
    Z_i=np.random.choice(2 # 0 or 1のどちらかを取得するという意味
                         , size=1 # 1つの値だけ取得する
                         , p=[1-z_prob[i], z_prob[i]] # 購入する確率(1が入る確率)がz_prob[i]
                        )[0]
    Z=np.append(Z, Z_i)

In [5]:
e_y=randn(num_data)

Y=-x_1 + 30*x_2 + 10*Z + 80 + 10*e_y

### データフレームにまとめる

In [6]:
df=pd.DataFrame({'年齢':x_1
                 , '性別':x_2
                 , 'CMを見た':Z
                 , '購入量':Y})

### 作成したデータの中身を確認

In [7]:
pd.concat(
    [df.count().rename('件数'),
    df.dtypes.rename('データ型'),
    df.nunique().rename('ユニーク件数'),
    df.isnull().sum().rename('null件数'),
    (df.isnull().sum() * 100 / df.shape[0]).rename('欠損の割合 (%)').round(2)],
    axis=1
)

,件数,データ型,ユニーク件数,null件数,欠損の割合 (%)
年齢,200,int32,58,0,0.0
性別,200,int32,2,0,0.0
CMを見た,200,float64,2,0,0.0
購入量,200,float64,200,0,0.0


# 結果

### 単純な購入量の平均値を比べる

In [8]:
print(df[df['CMを見た']==1].mean())
print('-------')
print(df[df['CMを見た']==0].mean())

年齢       55.836066
性別        0.483607
CMを見た     1.000000
購入量      49.711478
dtype: float64
-------
年齢       32.141026
性別        0.692308
CMを見た     0.000000
購入量      68.827143
dtype: float64


CMを見たほうが購入量が高いはずが、CMを見たほうが約20ほど低くなっている  
年齢・性別の共変量によりうまく効果を推定できていないことがわかる

### sklearnによる回帰分析

In [9]:
from sklearn.linear_model import LinearRegression

# 説明変数
X=df[['年齢', '性別', 'CMを見た']]
y=df['購入量']

reg=LinearRegression().fit(X, y)

print('係数：', reg.coef_)

係数： [-0.95817951 32.70149412 10.41327647]


CMを見たほうが約10高くなっており効果をうまく推定できていることがわかる

### statsmodelsによる回帰分析

In [10]:
import statsmodels.api as sm

x_add_const=sm.add_constant(X)
model_sm=sm.OLS(y,x_add_const).fit()
print(model_sm.summary())

                            OLS Regression Results                            
Dep. Variable:                    購入量   R-squared:                       0.814
Model:                            OLS   Adj. R-squared:                  0.811
Method:                 Least Squares   F-statistic:                     285.8
Date:                Tue, 09 Apr 2024   Prob (F-statistic):           2.70e-71
Time:                        11:29:20   Log-Likelihood:                -745.28
No. Observations:                 200   AIC:                             1499.
Df Residuals:                     196   BIC:                             1512.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         76.9845      2.125     36.221      0.0

sklearnと同じ結果になっている  
また、係数だけでなく係数の標準誤差・P値などの統計情報も表示される

### pingouinによる回帰分析

In [11]:
import pingouin as pg

pg.linear_regression(X,y)

,names,coef,se,T,pval,r2,adj_r2,CI[2.5%],CI[97.5%]
0,Intercept,76.984519,2.125428,36.220706,8.803100e-89,0.81394,0.811092,72.792874,81.176164
1,年齢,-0.958180,0.051670,-18.544147,5.339510e-45,0.81394,0.811092,-1.060080,-0.856279
2,性別,32.701494,1.506017,21.713895,4.733033e-54,0.81394,0.811092,29.731416,35.671572
3,CMを見た,10.413276,1.976754,5.267866,3.617649e-07,0.81394,0.811092,6.514838,14.311715


sklearn・statsmodelsと同じ結果になっている  
また、係数だけでなく係数の標準誤差・P値などの統計情報も表示される  
statsmodelsと異なる点は統計情報をテーブルデータで取得できる